In [ ]:
import numpy as np
import json
from scipy.sparse import load_npz
import os
from tqdm import tqdm

# FOLDERS = ["AIRWAY_HYPERREACTIVITY/", "ARTHRITIS/", 
#  "CROHNS/", "DIABETES_II/", "HEART_FAILURE/", "HIV/", "NEOPLASM_BREAST/"]

FOLDER = "CROHNS/"
DGIDB_DIRECTORY = "../Gen_Hypergraph/output/" + FOLDER
DGIDB_CONVERGED_VECTOR_PATH = "../Methods/output/" + FOLDER + "DGIDB_vector.npy"
MSIGDB_DIRECTORY = "../Gen_Hypergraph/output/MSigDB_FULL/"
restart_prob = 0.2  # Restart probability (theta)
num_iterations = 10  # Number of iterationsh

OUTPUT_FOLDER = "./robustnodes/DIABETES_II/"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

disease = "type 2 diabetes mellitus"



In [60]:
# Open the JSON file and load its content into a dictionary
with open(DGIDB_DIRECTORY + "gene_to_index.json", "r") as file:
    dgidb = json.load(file)
with open(MSIGDB_DIRECTORY + "gene_to_index.json", "r") as file:
    msigdb = json.load(file)

In [61]:
import numpy as np
# Jump probability for matching genes
w = 1

# Number of genes (assuming they are both of same size or matchable)
num_genes_dgidb = len(dgidb)
num_genes_msigdb = len(msigdb)

# Initialize the inter-layer matrix (D) with zeros
D = np.zeros((num_genes_dgidb, num_genes_msigdb))
i = 0
# Build the inter-layer matrix (D)
for gene_dgidb, idx_dgidb in dgidb.items():
    # If the gene exists in both gene-to-index mappings
    if gene_dgidb in msigdb:      
        idx_msigdb = msigdb[gene_dgidb]
        D[idx_dgidb, idx_msigdb] = w  # Set jump probability
        i += 1
rows_with_high_sum = np.where(D.sum(axis=1) > 0)[0]

In [62]:
# Load matrices
MSIGDB_weighted_matrix = load_npz(MSIGDB_DIRECTORY + "hypergraph_incidence_matrix_weighted.npz")
MSIGDB_binary_matrix = load_npz(MSIGDB_DIRECTORY + "hypergraph_incidence_matrix_binary.npz")
DGIDB_binary_matrix = load_npz(DGIDB_DIRECTORY + "hypergraph_incidence_matrix_binary.npz")
DGIDB_vector =  np.load(DGIDB_CONVERGED_VECTOR_PATH)

num_genes_MSIGDB = MSIGDB_binary_matrix.shape[0]  # Number of genes in MSIGDB

In [63]:
import torch
from tqdm import tqdm

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [64]:

dgidb_vector_complete = np.zeros(num_genes_MSIGDB)

for gene in tqdm(range(num_genes_MSIGDB), desc="DGIDB full vector calculation"):
    # Intra-hypergraph transitions in MSIGDB (moving within MSIGDB)
    connected_pathways = MSIGDB_binary_matrix[gene, :].nonzero()[1]  # Nonzero columns in MSIGDB binary matrix

    if len(connected_pathways) == 0:
        continue  # Skip if no pathways are found

    for pathway in connected_pathways:
        # Find genes connected to the selected pathway (weighted transition in MSIGDB)
        connected_genes = MSIGDB_weighted_matrix[:, pathway].toarray().flatten()
        neighbor_genes = np.where(connected_genes > 0)[0]  # Get genes with nonzero weight

        # Check if the current gene has a DGIDB connection
        dgidb_gene = np.where(D[:, gene] > 0)[0]  # Find DGIDB neighbors of the current MSIGDB gene
        if len(dgidb_gene) == 1:
            dgidb_drugs = DGIDB_binary_matrix[dgidb_gene[0], :].nonzero()[1]
            neighbor_genes_set = set()  # To avoid duplicates
        
            for drug in dgidb_drugs:
                # Get genes connected through the same drug (edge)
                connected_genes = DGIDB_binary_matrix[:, drug].toarray().flatten()
                neighbor_genes = np.where(connected_genes > 0)[0]
                # Add unique neighbors to the set
                neighbor_genes_set.update(neighbor_genes)

            # Sum contributions from unique DGIDB neighbors
            if len(neighbor_genes_set) > 0:
                neighbor_genes_list = list(neighbor_genes_set)
                dgidb_contribution = np.sum(DGIDB_vector[neighbor_genes_list])  # Sum unique contributions
                dgidb_vector_complete[gene] += dgidb_contribution  # Store in the complete vector

# Normalize dgidb_vector_complete to avoid overflow
dgidb_vector_complete /= np.sum(dgidb_vector_complete) if np.sum(dgidb_vector_complete) > 0 else 1
dgidb_vector_complete = torch.from_numpy(dgidb_vector_complete).to(device)

DGIDB full vector calculation: 100%|██████████| 21981/21981 [17:44<00:00, 20.64it/s]  


In [65]:
def get_hyper_randomwalk_torch(DGIDB_binary_matrix, DGIDB_vector, MSIGDB_weighted_matrix, MSIGDB_binary_matrix, D, restart_prob, num_iterations, P, dgidb_vector_complete, v0_sample):
    vi = v0_sample.clone()
    teleport_torch = v0_sample.clone()
    distance_list = []
    for k in range(num_iterations):
        print(f"Iteration {k + 1}")
        vj = vi.clone()

        # Vectorized transition step
        # Resulting shape will be (n, n)

        # Step 2: Normalize P by dividing by the sum of each column (with a small constant to avoid division by zero)
        P = P / (P.sum(dim=0, keepdim=True) + 1e-9)
        vi_new = torch.matmul(P.T.float(), vj.float())  # Transition from vj to vi_new using matrix multiplication

        # Normalize and combine
        vi_new = vi_new / (vi_new.sum() + 1e-9)
        
        vi = restart_prob * vi_new.T + (1 - restart_prob) * teleport_torch + dgidb_vector_complete

        distance = torch.sum(torch.abs(vj - vi)).item()
        distance_list.append(distance)


    importance_scores = torch.argsort(vi, descending=True)
    importance_values = vi[importance_scores]

    return {
        "Importance": list(zip(importance_scores.tolist(), importance_values.tolist())),
        "Distance": distance_list
    }


In [66]:
# result = get_hyper_randomwalk(DGIDB_binary_matrix, DGIDB_vector, MSIGDB_weighted_matrix, MSIGDB_binary_matrix, D, restart_prob, num_iterations)
# Print results
MSIGDB_weighted = torch.tensor(MSIGDB_weighted_matrix.toarray(), dtype=torch.float32, device=device)
MSIGDB_binary = torch.tensor(MSIGDB_binary_matrix.toarray(), dtype=torch.float32, device=device)
DGIDB_binary = torch.tensor(DGIDB_binary_matrix.toarray(), dtype=torch.float32, device=device)
# DGIDB_vector = torch.tensor(DGIDB_vector, dtype=torch.float32, device=device)
# D = torch.tensor(D, dtype=torch.float32, device=device)

num_genes_MSIGDB = MSIGDB_binary.shape[0]
v0 = torch.full((num_genes_MSIGDB,), 1.0 / num_genes_MSIGDB, dtype=torch.float32, device=device)
teleport = v0.clone()
P = MSIGDB_weighted @ MSIGDB_weighted.T
v0 = torch.full((num_genes_MSIGDB,), 1.0 / num_genes_MSIGDB, dtype=torch.float32, device=device)
result = get_hyper_randomwalk_torch(DGIDB_binary_matrix, DGIDB_vector, MSIGDB_weighted_matrix, MSIGDB_binary_matrix, D, restart_prob, num_iterations, P, dgidb_vector_complete, v0)
print("Top Indices by Importance:")
for index, score in result["Importance"][:10]:
    print(f"Index {index}: {score:.6f}")

print("\nDistance per Iteration:")
print(result["Distance"])

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Top Indices by Importance:
Index 13090: 0.094853
Index 9287: 0.093089
Index 14504: 0.073376
Index 10310: 0.072723
Index 12118: 0.066522
Index 9914: 0.046309
Index 12033: 0.045071
Index 6813: 0.044140
Index 14984: 0.039681
Index 9794: 0.033483

Distance per Iteration:
[1.0000007461530913, 0.09796130485483445, 0.0012409969931468368, 1.9727754988707602e-05, 4.050161805935204e-07, 5.4700649343430996e-08, 6.05832610744983e-08, 5.8225850807502866e-08, 4.721368895843625e-08, 4.757748683914542e-08]


In [67]:
import pandas as pd
# Load the JSON data from the file
with open(MSIGDB_DIRECTORY + 'gene_to_index.json', 'r') as file:
    gene_to_index = json.load(file)

# Invert the dictionary to map indices back to genes
index_to_gene = {v: k for k, v in gene_to_index.items()}
# human_gene2refseq = NCBI_INFO[NCBI_INFO['#tax_id'] == 9606]
# id_to_gene_claim = pd.Series(human_gene2refseq.Symbol.values, index=human_gene2refseq.GeneID).to_dict()
with open("../Methods/id_to_gene_claim.json", "r") as f:
    id_to_gene_claim = json.load(f)

def get_gene_claim_name(ncbi_gene_id):
    ncbi_gene_id = int(ncbi_gene_id)
    result = id_to_gene_claim[str(ncbi_gene_id)]
    if result:
        return result
    else:
        return "Gene name not found"

In [68]:
results_df = pd.DataFrame(result["Importance"], columns=['Index', 'Score'])
results_df["ncbi_gene_id"] = results_df["Index"].apply(index_to_gene.get)
results_df["claim_name"] = results_df["ncbi_gene_id"].apply(get_gene_claim_name)
results_df.to_csv(OUTPUT_FOLDER + "baseline.csv", index=False)

In [69]:
import torch
import numpy as np
import random
# Original uniform prior (used as mean of Dirichlet)
K = num_genes_MSIGDB

v0_base = torch.full((K,), 1.0 / K, dtype=torch.float32, device=device)
def perturb_matrix(matrix, fraction=0.1, amount=0.1, device=None):
    # Convert to dense NumPy array
    if hasattr(matrix, "toarray"):
        matrix_np = matrix.toarray()
    else:
        matrix_np = np.array(matrix)

    # Find non-zero indices
    nonzero_indices = np.argwhere(matrix_np > 0)
    num_edges = len(nonzero_indices)

    if num_edges == 0:
        raise ValueError("Matrix has no non-zero entries to perturb.")

    # Calculate perturbation value
    mean_weight = matrix_np[matrix_np > 0].mean()
    perturb_value = amount * mean_weight

    # Randomly select indices to perturb
    num_to_perturb = max(1, int(fraction * num_edges))
    selected_indices = random.sample(list(nonzero_indices), num_to_perturb)

    # Apply perturbation
    for i, j in selected_indices:
        matrix_np[i, j] += perturb_value

    # Convert to PyTorch tensor
    perturbed_tensor = torch.tensor(matrix_np, dtype=torch.float32, device=device)

    return perturbed_tensor



for i in range(20):
    # Convert sparse matrices to dense (or use PyTorch sparse support if memory limited)
    MSIGDB_binary = torch.tensor(MSIGDB_binary_matrix.toarray(), dtype=torch.float32, device=device)
    DGIDB_binary = torch.tensor(DGIDB_binary_matrix.toarray(), dtype=torch.float32, device=device)
    # DGIDB_vector = torch.tensor(DGIDB_vector, dtype=torch.float32, device=device)
    # D = torch.tensor(D, dtype=torch.float32, device=device)

    num_genes_MSIGDB = MSIGDB_binary.shape[0]

    MSIGDB_weighted_perturbed = perturb_matrix(MSIGDB_weighted_matrix, fraction=0.1, amount=1, device=device)
    P_PERTURBED = MSIGDB_weighted_perturbed @ MSIGDB_weighted_perturbed.T
    diff = torch.norm(P_PERTURBED - P, p='fro')  # Frobenius norm
    print("Frobenius norm difference between P_PERTURBED and P:", diff.item())

    # Call your method with sampled v0
    result = get_hyper_randomwalk_torch(
        DGIDB_binary_matrix,
        DGIDB_vector,
        MSIGDB_weighted_matrix,
        MSIGDB_binary_matrix,
        D,
        restart_prob,
        num_iterations,
        P_PERTURBED,
        dgidb_vector_complete,
        v0_base
    )
    results_df = pd.DataFrame(result["Importance"], columns=['Index', 'Score'])
    results_df["ncbi_gene_id"] = results_df["Index"].apply(index_to_gene.get)
    results_df["claim_name"] = results_df["ncbi_gene_id"].apply(get_gene_claim_name)
    results_df.to_csv(OUTPUT_FOLDER + f"robustnodes_{i}.csv", index=False)

Frobenius norm difference between P_PERTURBED and P: 0.8921632766723633
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Frobenius norm difference between P_PERTURBED and P: 0.8949627876281738
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Frobenius norm difference between P_PERTURBED and P: 0.8935789465904236
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Frobenius norm difference between P_PERTURBED and P: 0.8974792957305908
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Frobenius norm difference between P_PERTURBED and P: 0.897468626499176
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Frobenius norm difference between P_

In [70]:
import pandas as pd
df = pd.read_csv("../Data/df_score_as_value.tsv", sep="\t", index_col=0)

In [71]:
filtered_df = df[df[disease].notna()]
filtered_df = filtered_df.sort_values(by=disease, ascending=False)

In [72]:
def ranking_agreement_score(L1_scores: dict, L1_ranking: list, L2_ranking: list, k: int = None) -> float:
    if k is None:
        k = len(L2_ranking)
    
    total = 0
    # Assign proxy scores to L2 based on inverse rank (1-based)
    for idx, item in enumerate(L2_ranking[:k]):
        if item not in L1_scores:
            continue  # skip items not in L1
        
        score_L1 = L1_scores[L1_ranking[idx]]
        score_L2 = L1_scores[item] 

        if score_L1 == 0 or score_L2 == 0:
            continue  # avoid division by zero
        ratio = min(score_L2 / score_L1, score_L1 / score_L2)
        total += ratio

    return total / k

In [73]:
import re
baseline = pd.read_csv(os.path.join(OUTPUT_FOLDER, "baseline.csv"))
baselineGenes = baseline["claim_name"][:10]

pattern = re.compile(r"robustnodes_(\d+)\.csv")
csv_files = [f for f in os.listdir(OUTPUT_FOLDER) if pattern.match(f)]
wmra_scores = []
k = 10

for fname in tqdm(sorted(csv_files, key=lambda f: int(pattern.match(f).group(1)))):
    drichletNum = int(pattern.match(fname).group(1))
    drichletSample = pd.read_csv(os.path.join(OUTPUT_FOLDER, fname))

    WMRA = ranking_agreement_score(filtered_df[disease], filtered_df[disease].index, drichletSample["claim_name"][:k], k)
    wmra_scores.append(WMRA)


print(wmra_scores)

100%|██████████| 20/20 [00:00<00:00, 96.76it/s]

[0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576, 0.16846947565732576]


In [74]:
import statistics

mean = statistics.mean(wmra_scores)
stdev = statistics.stdev(wmra_scores)

print(f"Mean: {mean:.4f}, Std Dev: {stdev:.4f}")
output_path = os.path.join(OUTPUT_FOLDER, "final.txt")
with open(output_path, "w") as f:
    f.write(f"Mean: {mean:.4f}, Std Dev: {stdev:.4f}\n")

Mean: 0.1685, Std Dev: 0.0000
